In [ ]:
import json
import pandas as pd
import random
p= .1#% of the lines
#keep the header, then take only 1% of lines
#if random from [0,1] interval is greater than 0.01 the row will be skipped
raw_data_with_measures = pd.read_csv(
         '/content/data.csv', 
         skiprows=lambda i: i>0 and random.random() > p)

In [ ]:

# makes the string lists in artists column into actual lists
raw_data_with_measures['artists'] = raw_data_with_measures['artists'].apply(lambda x: x[1:-1].split(', ')) 

# opens the lists so that each artist is on a song has his own row with the song
raw_data_with_measures = raw_data_with_measures.explode('artists') 

#  we want to take off the quotes on the artists
raw_data_with_measures['artists'] = raw_data_with_measures['artists'].apply(lambda x: x.strip("'")) 

 # we will just focus on these columns for now, and rename our data to raw_data
raw_data =  raw_data_with_measures.loc[:,['artists','id','name','popularity','year']]



raw_data

In [ ]:
raw_data_genre = pd.read_csv('data_w_genres.csv')
raw_data_genre.head(4)

In [ ]:
# makes it into actual list instead of string list
raw_data_genre['genres'] = raw_data_genre['genres'].apply(lambda x: x[1:-1].split(', '))  

#filters out empty values in each list    
for i in raw_data_genre.index:

    raw_data_genre['genres'].loc[i] = list(filter(None, raw_data_genre['genres'][i]))  

raw_data_genre.head(10)

In [ ]:
#grab artist with no genre 
missing_genre_artists = raw_data_genre[raw_data_genre['genres'].map(lambda x: len(x)) < 1]
missing_genre_artists

In [ ]:
missing_genre_artists_split = missing_genre_artists.copy()

# split into words 
missing_genre_artists_split['artists'] = missing_genre_artists_split['artists'].apply(lambda x: x.split())  
missing_genre_artists_split

In [ ]:
# lets find most common words in artists

def word_count(data):

    all_words = []  
    for elmnt in data['artists']:  
        all_words += elmnt  

    val_counts = pd.Series(all_words).value_counts()

    return val_counts.head(40)

word_count(missing_genre_artists_split)

In [ ]:
pd.options.mode.chained_assignment = None 

#for loop to create new genres for artists that are missing genres by appending word in their name to genre 
for i in raw_data_genre.index:
    
    if 'Broadway' in raw_data_genre['artists'].loc[i]:
        raw_data_genre['genres'].loc[i].append("'broadway'")
    if 'Motion Picture' in raw_data_genre['artists'].loc[i]:
        raw_data_genre['genres'].loc[i].append("'movie tunes'")
    if 'Cast' in raw_data_genre['artists'].loc[i]:
        raw_data_genre['genres'].loc[i].append("'ensemble'")      
    if 'Ensemble' in raw_data_genre['artists'].loc[i]:
        raw_data_genre['genres'].loc[i].append("'ensemble'")  
    if 'Orchestra' in raw_data_genre['artists'].loc[i]:
        raw_data_genre['genres'].loc[i].append("'orchestra'")
    if 'Orquesta' in raw_data_genre['artists'].loc[i]:             
        raw_data_genre['genres'].loc[i].append("'orchestra'")
    if 'Orchestre' in raw_data_genre['artists'].loc[i]:
        raw_data_genre['genres'].loc[i].append("'orchestra'")
    if 'Chorus' in raw_data_genre['artists'].loc[i]:
        raw_data_genre['genres'].loc[i].append("'chorus'")
    if 'Choir' in raw_data_genre['artists'].loc[i]:
        raw_data_genre['genres'].loc[i].append("'choir'")
        
    raw_data_genre['genres'].loc[i] = list(dict.fromkeys(raw_data_genre['genres'].loc[i])) # removes duplicates
    

raw_data_genre.head(12)

In [ ]:
#create final df that removes artists without genres 
final_data_genre = raw_data_genre[raw_data_genre['genres'].map(lambda x: len(x)) > 0] # removes empty lists
final_data_genre = final_data_genre.reset_index(drop=True)
final_data_genre = final_data_genre.drop('count', axis =1 )
final_data_genre.rename(columns={'popularity': 'Artist Popularity'}, inplace=True)
final_data_genre['Artist Popularity'] = final_data_genre['Artist Popularity'].astype(int)
final_data_genre

In [ ]:
#finally merge two dataframes for final df 
merged_df = raw_data.merge(final_data_genre, how = 'inner', on = ['artists'])
merged_df_copy = merged_df.copy()
merged_df_copy.rename(columns={'artists': 'Artist', 'name':'Song Name','popularity':'Song Popularity','year':'Year','genres':'Genres'}, inplace=True)
merged_df_copy['Genres_cleaned'] = [','.join(map(str, l)) for l in merged_df_copy['Genres']]
merged_df_copy.drop(columns=['Genres'], inplace=True)
final_df = merged_df_copy.drop(columns=['key', 'mode'])


In [ ]:
#dummy encode genres 
new_df = pd.concat([final_df.drop('Genres_cleaned', 1), final_df['Genres_cleaned'].str.get_dummies(sep=",")], 1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
artist_encoder = LabelEncoder()
song_encoder = LabelEncoder()
new_df['Artist'] = artist_encoder.fit_transform(new_df['Artist'])
new_df['Song Name'] = song_encoder.fit_transform(new_df['Song Name'])

In [5]:
new_df

,Unnamed: 0,Artist,id,Song Name,Song Popularity,Year,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,Artist Popularity,"""canadian children's music""","""canzone d'autore""","""children's folk""","""children's music""","""death 'n' roll""","""punk 'n' roll""",'a cappella','abstract beats','abstract hip hop','abstract','acid house','acid rock','acousmatic','acoustic blues','acoustic pop','acoustic punk','adult standards','adventista','afghan pop','african percussion','african reggae','african rock','afro dancehall',...,'virgin islands reggae','virginia metal','virginia punk','vocal ensemble','vocal harmony group','vocal house','vocal jazz','washington indie','water','welsh indie','welsh metal','welsh rock','west coast rap','west coast reggae','west coast trap','west end','west virginia indie','western americana','western saharan folk','western swing','white noise','wonky','world fusion','world worship','world','worship','wrestling','wu fam','wyoming indie','wyoming roots','xhosa','yacht rock','ye ye','yiddish folk','yodeling','york indie','zhongguo feng','zolo','zouk','zydeco'
0,0,1948,73LY41HCJzlQwoNBmWM7Md,14747,0,1928,0.984176,0.660092,177021.390921,0.287901,5.356324e-01,0.198782,-12.027960,0.110733,122.819144,0.746016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1948,4285vY2v7Z1Kyh3Gi3sDrz,2270,0,1930,0.984176,0.660092,177021.390921,0.287901,5.356324e-01,0.198782,-12.027960,0.110733,122.819144,0.746016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,1948,2KeiqgScES5dOL8E6XjOfZ,10003,0,1940,0.984176,0.660092,177021.390921,0.287901,5.356324e-01,0.198782,-12.027960,0.110733,122.819144,0.746016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,1948,2mXu9biCofU87odRNcJCQr,7301,0,1945,0.984176,0.660092,177021.390921,0.287901,5.356324e-01,0.198782,-12.027960,0.110733,122.819144,0.746016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,1948,7E9qmGwQA7Bo06tKHRF852,2414,0,1928,0.984176,0.660092,177021.390921,0.287901,5.356324e-01,0.198782,-12.027960,0.110733,122.819144,0.746016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19495,19495,6539,5vDCp6yZAikAoJ6NOEEFdp,3088,67,2019,0.244750,0.673500,179433.250000,0.669750,6.025000e-05,0.135550,-7.808250,0.124475,108.497000,0.456750,67,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19496,19496,3166,7sd72KZS8D59g5NmhxyHpJ,13072,72,2019,0.082133,0.889333,204385.000000,0.665667,1.846667e-05,0.078767,-7.549000,0.196633,100.299333,0.826667,71,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19497,19497,4102,54JxxJ2KW8nCeb6ZnroLA0,10614,69,2020,0.298652,0.692091,172979.666667,0.642121,7.190909e-07,0.130121,-5.798848,0.040312,109.848879,0.677273,68,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19498,19498,1808,5dAtKXyrQoQW054LZqJ5dP,3959,67,2020,0.001360,0.391750,234245.000000,0.853000,3.798300e-01,0.185075,-3.796750,0.218050,135.168500,0.201000,55,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [11]:
col_name = "id"
first_col = new_df.pop(col_name)
new_df.insert(0, col_name, first_col)

In [ ]:
new_df

In [13]:
import pandas as pd 
from sklearn.neighbors import NearestNeighbors
#knn fit then predict 
knn = NearestNeighbors(n_neighbors=1000)
knn.fit(new_df[new_df.columns[2:]])

def predict(track_id):

    id = new_df.index[new_df['id'] == track_id]
    series = new_df.iloc[id, 2:].to_numpy()

    neighbors = knn.kneighbors(series)
    new_obs = neighbors[1][0][6:20]
    return list(new_df.loc[new_obs, 'id'])

In [17]:
predict('2FVMbLuIxo0dz3KLXgN6Jh')

['6a0arm7sfmKWF30xWqAL9D',
 '38P4YlPZabEM1GAhWT9FOq',
 '1eF0kl9Z76WsGAK6yEewQK',
 '5EpuJNMQ6LkPcidRR1P6D4',
 '67S1POEmchWHyss0nboDDN',
 '5GV4kQCF8eBuY8LPe9FBDu',
 '2rG87zMRSfsfcPOOUAu99p',
 '0rc9jL9G6RQHtgvIDYUmLD',
 '599NflshA2ltPAxvihhR0z',
 '1CymG1KJAhOoktmo0FRMlY',
 '7yXsf6EGWOm3dj2gZfmKgj',
 '6Ai8e94kxYZah61XAsJsiu',
 '1kGgPrkaBG4o2D2Cemsvg3',
 '7DKjnQH7yeCUPXX88Hh12p']

In [ ]:
#pickle 
import joblib

joblib.dump(knn, "KNN_final.joblib", compress=2, protocol=None, cache_size=None)


['KNN_final.joblib']